In [2]:
# Importing bigqueery
from google.cloud import bigquery

In [4]:
# Authenticating the user
from google.colab import auth
auth.authenticate_user()
print("Authenticated")


Authenticated


In [5]:
# Reference to my project id so that i can use .Client()
# Creating a client object
project_id = "sqlproject-302612"
client = bigquery.Client(project=project_id)

In [7]:
# Fetching the data
dataset_ref = client.dataset("crypto_bitcoin", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)


In [8]:
# Listing tables
tables = client.list_tables(dataset)
for table in tables:
  print(table.table_id)

blocks
inputs
outputs
transactions


In [9]:
# Investigating Transactions table
# Question: How many bitcoins transactions are made per month?

# Firt, let's get the transaction table
table_ref = dataset_ref.table("transactions")
table = client.get_table(table_ref)

In [11]:
# Checking the data
client.list_rows(table, max_results=5).to_dataframe()

,hash,size,virtual_size,version,lock_time,block_hash,block_number,block_timestamp,block_timestamp_month,input_count,output_count,input_value,output_value,is_coinbase,fee,inputs,outputs
0,f5d9499fb93c104c30f2f0a6055787c4e788b0fec8be89...,512,512,1,0,000000000000000160f3b852402569f6a1f1b38cd1a35a...,273029,2013-12-04 13:32:16+00:00,2013-12-01,2,6,978848094,978798000,False,50094,"[{'index': 0, 'spent_transaction_hash': '0608f...","[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
1,de43728cb2982ab848938d061f55d522e15fe79bb5c3ab...,512,512,1,0,00000000000000003a1260f6a07fa8a9f66bbc89972641...,273014,2013-12-04 11:08:18+00:00,2013-12-01,2,6,633984346,633934346,False,50000,"[{'index': 0, 'spent_transaction_hash': '10e0e...","[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
2,b9aaa4ab930570f8af4eda306637b9a96b6a5a8bba0b15...,768,768,1,0,0000000000000003696f88dc5eb21e54f25d94938ad562...,277416,2013-12-28 14:48:01+00:00,2013-12-01,4,1,170617369,170607369,False,10000,"[{'index': 0, 'spent_transaction_hash': '91ded...","[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
3,71863169438b978b5cc812261781a333a2a20b03509601...,768,768,1,0,000000000000000356bdf67fc717f56f065935ec29edd0...,272883,2013-12-03 18:47:25+00:00,2013-12-01,4,2,76686838,76586838,False,100000,"[{'index': 0, 'spent_transaction_hash': 'f9b6b...","[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."
4,eef7f497635bd52bf89393aff898bef8a93dce7058c9ee...,768,768,1,0,000000000000000574f53b20f90356afb560c8e68587f7...,272702,2013-12-02 19:11:15+00:00,2013-12-01,4,4,65856646,65816646,False,40000,"[{'index': 0, 'spent_transaction_hash': '54f5d...","[{'index': 0, 'script_asm': 'OP_DUP OP_HASH160..."


In [25]:
#Construction the query
query = ''' WITH time AS(SELECT DATE(block_timestamp) AS month                                         
                                  FROM `bigquery-public-data.crypto_bitcoin.transactions`                                 
                                 )
            SELECT month, COUNT(month) AS n_transaction
            FROM time
            GROUP BY month
            ORDER BY month
        '''

In [26]:
#Submitting the query
query_job = client.query(query)
n_transactions_month = query_job.to_dataframe()
n_transactions_month.head()

,month,n_transaction
0,2009-01-03,1
1,2009-01-09,14
2,2009-01-10,61
3,2009-01-11,93
4,2009-01-12,101
